Defining global path in python. These variables will be stored after one execution

In [1]:
SRC='/home/jmurga/mktComparison/scripts/src'
DATA='/home/jmurga/mktComparison/rawData/dmel'
FASTAS='/data/shared/dgn'

Required libraries

In [15]:
import os
import re
import sys
import numpy as np
import pandas as pd
import pyfaidx as px
from numpy import array 

Ad-hoc scripts

In [3]:
sys.path.insert(0, SRC)
from reverseComplement import reverseComplement
from degenerancy import degenerate
from foldPositions import foldPositions
from nDistribution import nDistribution

### Drosophila melanogaster genes 

To execute bash code we created a snippets through nbextensions containing the following paths in order to avoid copy and paste cells. %%bash magic do not recognize previous variables. Adding mkdir command to create necesary paths

In [4]:
%%bash
DATA='/home/jmurga/mktComparison/rawData/dmel/annotations'
BASIC='/home/jmurga/mktComparison/rawData/dmel/annotations/basicAnnotation'
CDS='/home/jmurga/mktComparison/rawData/dmel/annotations/cds'
GENES='/home/jmurga/mktComparison/rawData/dmel/annotations/genes'
TEMPORAL='/home/jmurga/mktComparison/rawData/dmel/annotations/tmp'
ALLELEFREQ='/home/jmurga/mktComparison/rawData/dmel/alleleFrequencies'

mkdir -p ${DATA}
mkdir -p ${BASIC}
mkdir -p ${CDS}
mkdir -p ${GENES}
mkdir -p ${TEMPORAL}
mkdir -p ${ALLELEFREQ}

#### Download Flybase annotation

In [5]:
%%bash
DATA='/home/jmurga/mktComparison/rawData/dmel/annotations'

cd ${DATA}
# Flybase annotation
wget ftp://ftp.flybase.net/genomes/Drosophila_melanogaster/dmel_r5.57_FB2014_03/gff/dmel-all-no-analysis-r5.57.gff.gz
gunzip dmel-all-no-analysis-r5.57.gff.gz

--2020-01-03 14:45:15--  ftp://ftp.flybase.net/genomes/Drosophila_melanogaster/dmel_r5.57_FB2014_03/gff/dmel-all-no-analysis-r5.57.gff.gz
           => ‘dmel-all-no-analysis-r5.57.gff.gz’
Resolving ftp.flybase.net (ftp.flybase.net)... 52.23.126.124
Connecting to ftp.flybase.net (ftp.flybase.net)|52.23.126.124|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /genomes/Drosophila_melanogaster/dmel_r5.57_FB2014_03/gff ... done.
==> SIZE dmel-all-no-analysis-r5.57.gff.gz ... 100487286
==> PASV ... done.    ==> RETR dmel-all-no-analysis-r5.57.gff.gz ... done.
Length: 100487286 (96M) (unauthoritative)

     0K .......... .......... .......... .......... ..........  0%  165K 9m53s
    50K .......... .......... .......... .......... ..........  0%  244K 8m17s
   100K .......... .......... .......... .......... ..........  0% 55,5M 5m32s
   150K .......... .......... .......... .......... ..........  0% 41,6M 4m9s


#### Parsing and cleaning flybase annotation to execute all operations by chr
Operate by chr is faster due to grep on smaller files. Each folder contain an specific file foreach chromosome. 

In [ ]:
%%bash
DATA='/home/jmurga/mktComparison/rawData/dmel/annotations'
BASIC='/home/jmurga/mktComparison/rawData/dmel/annotations/basicAnnotation'
CDS='/home/jmurga/mktComparison/rawData/dmel/annotations/cds'
GENES='/home/jmurga/mktComparison/rawData/dmel/annotations/genes'
TEMPORAL='/home/jmurga/mktComparison/rawData/dmel/annotations/tmp'

mkdir -p ${BASIC}
mkdir -p ${CDS}
mkdir -p ${GENES}
mkdir -p ${TEMPORAL}

# Extract annotations from gff file
sed -e '/^>/,$d' ${DATA}/dmel-all-no-analysis-r5.57.gff | sed -e '/^2LHet/d' -e  '/2RHet/d' -e '/3LHet/d' -e '/3RHet/d' -e  '/^4\t/d' -e '/dmel_mitochondrion_genome/d' -e '/^U\t/d' -e '/^Uextra\t/d' -e  '/^XHet\t/d' -e '/^YHet\t/d' -e'/\tCG/d' > ${DATA}/dmelFiltered.gff
# Extract genes information
grep -P "\tgene\t" ${DATA}/dmelFiltered.gff  > ${DATA}/dmelFilteredGenes.gff
# Extract CDS information. Only protein coding genes
grep -P "\tCDS\t" ${DATA}/dmelFiltered.gff  > ${DATA}/dmelFilteredCds.gff
# Coding gene list. gene_id always on column 9, 3th field
cut -f1,9 ${DATA}/dmelFilteredCds.gff | tr ';' '\t' | cut -f1,2 | sort -u | sort -k1,1 > ${DATA}/codingGeneList.txt


CHR=( 2L 2R 3L 3R X )
# Parse gff file by chr
for chrNumber in "${CHR[@]}"
do
    grep -P "${chrNumber}\t"  ${DATA}/dmelFilteredGenes.gff | sort -k4,4n > ${BASIC}/gencode.v27lift37.basic.annotation.chr${chrNumber}.gff3 
    grep -P "${chrNumber}\t"  ${DATA}/dmelFilteredCds.gff | sort -k1,1 -k4,4n > ${CDS}/dmelFilteredCdsChr${chrNumber}.gff3 
    grep -P "${chrNumber}\t"  ${DATA}/dmelFilteredGenes.gff | sort -k1,1 -k4,4n > ${GENES}/dmelFilteredGenesChr${chrNumber}.gff3 
done

rm ${DATA}/dmel-all-no-analysis-r5.57.gff

#### Basic cleaned gene file
This file will include information about chromosomes, start coordinates, end coordinates strand, gene id and gene name. It will be and perform calculations on gene coordinates. Kind of gff file easier to work with

In [ ]:
%%bash
DATA='/home/jmurga/mktComparison/rawData/dmel/annotations'
GENES='/home/jmurga/mktComparison/rawData/dmel/annotations/genes'
touch ${DATA}/flybaseGenesCleaned.tab 
printf "chr\tstartGene\tendGene\tstrand\tid\tname\n" > ${DATA}/flybaseGenesCleaned.tab

time while read LINE;
do 
    CHR=$(echo ${LINE} | cut -d' ' -f1)
    GENE=$(echo ${LINE} | cut -d' ' -f2 | sed 's/-cds//g')
    # echo $GENE

    fgrep `echo "${GENE};"`  ${GENES}/dmelFilteredGenesChr${CHR}.gff3 | fgrep ${CHR} |  cut -f1,4,5,7,9 | tr ';' '\t' | cut -f1,2,3,4,5,6 
    
done < ${DATA}/codingGeneList.txt | tr ' ' '\t' | sed 's/ID=//g' | sed 's/Name=//g' | sort -k1,1 -k2,2n >> ${DATA}/flybaseGenesCleaned.tab

In [8]:
dfGenes = pd.read_csv(DATA + '/annotations/flybaseGenesCleaned.tab',header = 0,sep='\t')
dfGenes.head()

,chr,startGene,endGene,strand,id,name
0,2L,7529,9484,+,FBgn0031208,CG11023
1,2L,9839,21376,-,FBgn0002121,l(2)gl
2,2L,21823,25155,-,FBgn0031209,Ir21a
3,2L,25402,65404,-,FBgn0051973,Cda5
4,2L,66482,71390,+,FBgn0067779,dbr


File to merge features annotated with gene name instead of gene id

In [9]:
idName = dfGenes[['id','name']]
idName.to_csv(DATA + '/annotations/idName.tab',sep='\t',index=False,header=True)

### CDS *D. melanogaster* degenerancy 

#### Raw cds coordinates

In [ ]:
%%bash
DATA='/home/jmurga/mktComparison/rawData/dmel/annotations'
CDS='/home/jmurga/mktComparison/rawData/dmel/annotations/cds'
TEMPORAL='/home/jmurga/mktComparison/rawData/dmel/annotations/tmp'

touch ${DATA}/cdsCoordinates.tab
printf "name\tchr\ttranscript\ttranscriptSize\tcoordinates\n" > ${DATA}/cdsCoordinates.tab

count=0
time tail -n+2 ${DATA}/flybaseGenesCleaned.tab | while read LINE;
do 
    echo '*************'    
    CHR=$(echo ${LINE} | cut -d' ' -f1)
    GENE=$(echo ${LINE} | cut -d' ' -f6)
    
    printf "${GENE} ${CHR}"
    
    fgrep `echo "Name=${GENE}-cds;"` ${CDS}/dmelFilteredCdsChr${CHR}.gff3 | grep -P "${CHR}\t" | cut -f9 | tr ';' '\n' | fgrep Parent | sort -u | tr ',' '\n' | cut -d'=' -f2 | sort -u > ${TEMPORAL}/transcriptTmp.tab

    while read transcript; do fgrep ${transcript} ${CDS}/dmelFilteredCdsChr${CHR}.gff3 | awk '{print $4,$5,$5-$4}' | awk -v gene="${GENE}" -v chr="${CHR}" -v transcript="$transcript" '{sum+=$1} {printf $1","$2","} END{print "\t"gene"\t"chr"\t"transcript"\t"sum}' | awk '{print $2,$3,$4,$5,$1}' | sed 's/,$//' |tr ' ' '\t' >> ${DATA}/cdsCoordinates.tab ;done < ${TEMPORAL}/transcriptTmp.tab 
    
    (( count++ ))

done

#### Check degenerancy by position

**Checking degenerancy by positions taking into account all transcripts and genes independently**  
Recoding CDS sequences to get 0fold, 2fold, 3fold and 4fold positions by transcript and genes, in order to estimate frequencies and divergence by type of functional sites.

In [18]:
sys.path.insert(0, SRC)
from reverseComplement import reverseComplement
from degenerancy import *

In [11]:
dfGenes = pd.read_csv(DATA + '/annotations/flybaseGenesCleaned.tab',header = 0,usecols=['id','name','chr','strand'],sep='\t')
cds = pd.read_csv(DATA + '/annotations/cdsCoordinates.tab',header=0,sep='\t')
cds = pd.merge(cds, dfGenes,  how='inner', left_on=['chr','name'], right_on = ['chr','name'])

In [12]:
def foldPositions(x):
    if('0' in x):
        return('0fold')
    elif('4' in x and '0' not in x and '2' not in x and '3' not in x):
        return('4fold')
    elif('2' in x and '0' not in x and '4' not in x and '3' not in x):
        return('2fold')
    elif('2' not in x and '0' not in x and '4' not in x) :
        return('3fold')
    else:
        return('mixed')

In [ ]:
import time
# for index, row in cds.iterrows():
df = pd.DataFrame()
# for geneId in np.unique(cds[(cds['id']=='FBgn0000028')].id):
for j in range(0,len(np.unique(cds.id))):
    start_time = time.time()
   
    geneId = np.unique(cds.id)[j]
#     geneId = geneId
    df = pd.DataFrame()
    print(geneId)
    
    for index, row in cds[(cds['id']==geneId)].iterrows():
        
        # Rewrite file each execution
        degen = []
        
        chrFile = px.Fasta(FASTAS + '/ref/Chr' + row['chr'] +'.fasta',sequence_always_upper=True)

        # Convert CDS list into numeric array
        coordinates = np.array(row['coordinates'].split(',')).astype(int).tolist()
        coordinates =  [coordinates[i:i+2] for i in range(0, len(coordinates), 2)]

        # Extract all CDS positions in a list in order to merge with degenerate sequences (same length -> same index)
        positions=[]
        for i in range(0,len(coordinates),1):
            positions.append(list(range(coordinates[i][0],coordinates[i][1]+1)))  
        allPositions = [item for sublist in positions for item in sublist]
        allPositions = np.asarray(allPositions)

        # Extract cds sequences
        seq = chrFile.get_spliced_seq(row['chr'], coordinates).seq
        if(row['strand'] == '-'):
            seq = reverseComplement(seq)
            allPositions = allPositions[::-1]
        if((len(seq)/3).is_integer() and seq[0:3]=='ATG'):
            # Append in list 
            m = degenerateFullPositions(seq)
            degen.append(list(m))
            nchr = np.array([row['chr']] * len(m))
            
            degen = [item for sublist in degen for item in sublist]
            degen = np.asarray(degen)

            data =  {'CHROM':nchr,'POS':allPositions,'degen':degen}
            tmp = pd.DataFrame(data)

            df = df.append(tmp)
            df = df.groupby(['CHROM','POS']).agg({'degen':','.join}).reset_index()

    #Save df
    df.to_csv(DATA + '/annotations/degeneracyDmelPositions.tab',mode='a',index=False,header=False,sep='\t')
    print(j,geneId,"--- %s seconds ---" % (time.time() - start_time))

FBgn0000008
0 FBgn0000008 --- 0.3305058479309082 seconds ---
FBgn0000014
1 FBgn0000014 --- 0.14184045791625977 seconds ---
FBgn0000015
2 FBgn0000015 --- 0.20324230194091797 seconds ---
FBgn0000017
3 FBgn0000017 --- 0.9174928665161133 seconds ---
FBgn0000018
4 FBgn0000018 --- 0.053694963455200195 seconds ---
FBgn0000022
5 FBgn0000022 --- 0.037393808364868164 seconds ---
FBgn0000024
6 FBgn0000024 --- 0.13294506072998047 seconds ---
FBgn0000028
7 FBgn0000028 --- 0.3640613555908203 seconds ---
FBgn0000032
8 FBgn0000032 --- 0.07344794273376465 seconds ---
FBgn0000036
9 FBgn0000036 --- 0.12050199508666992 seconds ---
FBgn0000037
10 FBgn0000037 --- 0.11948323249816895 seconds ---
FBgn0000038
11 FBgn0000038 --- 0.08182263374328613 seconds ---
FBgn0000039
12 FBgn0000039 --- 0.09658217430114746 seconds ---
FBgn0000042
13 FBgn0000042 --- 0.1582803726196289 seconds ---
FBgn0000043
14 FBgn0000043 --- 0.04466700553894043 seconds ---
FBgn0000044
15 FBgn0000044 --- 0.04935932159423828 seconds ---
FBgn

133 FBgn0000411 --- 0.11486506462097168 seconds ---
FBgn0000412
134 FBgn0000412 --- 0.09749245643615723 seconds ---
FBgn0000413
135 FBgn0000413 --- 0.16740846633911133 seconds ---
FBgn0000414
136 FBgn0000414 --- 0.888822078704834 seconds ---
FBgn0000416
137 FBgn0000416 --- 0.1376936435699463 seconds ---
FBgn0000422
138 FBgn0000422 --- 0.11124825477600098 seconds ---
FBgn0000426
139 FBgn0000426 --- 0.0329279899597168 seconds ---
FBgn0000427
140 FBgn0000427 --- 0.52447509765625 seconds ---
FBgn0000439
141 FBgn0000439 --- 0.0532679557800293 seconds ---
FBgn0000442
142 FBgn0000442 --- 0.06608200073242188 seconds ---
FBgn0000447
143 FBgn0000447 --- 0.06842494010925293 seconds ---
FBgn0000448
144 FBgn0000448 --- 0.35802197456359863 seconds ---
FBgn0000449
145 FBgn0000449 --- 0.05015158653259277 seconds ---
FBgn0000451
146 FBgn0000451 --- 0.18860578536987305 seconds ---
FBgn0000454
147 FBgn0000454 --- 0.13752102851867676 seconds ---
FBgn0000455
148 FBgn0000455 --- 0.07859539985656738 seconds 

FBgn0001168
264 FBgn0001168 --- 0.0688025951385498 seconds ---
FBgn0001169
265 FBgn0001169 --- 0.23994183540344238 seconds ---
FBgn0001170
266 FBgn0001170 --- 0.048456668853759766 seconds ---
FBgn0001174
267 FBgn0001174 --- 0.03537702560424805 seconds ---
FBgn0001179
268 FBgn0001179 --- 0.11126518249511719 seconds ---
FBgn0001180
269 FBgn0001180 --- 0.1117091178894043 seconds ---
FBgn0001185
270 FBgn0001185 --- 0.0763702392578125 seconds ---
FBgn0001186
271 FBgn0001186 --- 0.11859250068664551 seconds ---
FBgn0001187
272 FBgn0001187 --- 0.04776573181152344 seconds ---
FBgn0001189
273 FBgn0001189 --- 0.0870814323425293 seconds ---
FBgn0001197
274 FBgn0001197 --- 0.049230098724365234 seconds ---
FBgn0001202
275 FBgn0001202 --- 0.06170964241027832 seconds ---
FBgn0001206
276 FBgn0001206 --- 0.10443735122680664 seconds ---
FBgn0001208
277 FBgn0001208 --- 0.1434633731842041 seconds ---
FBgn0001215
278 FBgn0001215 --- 0.16908526420593262 seconds ---
FBgn0001216
279 FBgn0001216 --- 0.127415895

FBgn0002579
394 FBgn0002579 --- 0.09126472473144531 seconds ---
FBgn0002590
395 FBgn0002590 --- 0.08599138259887695 seconds ---
FBgn0002592
396 FBgn0002592 --- 0.041526079177856445 seconds ---
FBgn0002593
397 FBgn0002593 --- 0.050501346588134766 seconds ---
FBgn0002607
398 FBgn0002607 --- 0.052941322326660156 seconds ---
FBgn0002609
399 FBgn0002609 --- 0.038304805755615234 seconds ---
FBgn0002622
400 FBgn0002622 --- 0.054613590240478516 seconds ---
FBgn0002626
401 FBgn0002626 --- 0.07256650924682617 seconds ---
FBgn0002629
402 FBgn0002629 --- 0.03505349159240723 seconds ---
FBgn0002631
403 FBgn0002631 --- 0.03537893295288086 seconds ---
FBgn0002632
404 FBgn0002632 --- 0.029264211654663086 seconds ---
FBgn0002633
405 FBgn0002633 --- 0.036969661712646484 seconds ---
FBgn0002638
406 FBgn0002638 --- 0.1190798282623291 seconds ---
FBgn0002641
407 FBgn0002641 --- 0.11270785331726074 seconds ---
FBgn0002643
408 FBgn0002643 --- 0.3267850875854492 seconds ---
FBgn0002645
409 FBgn0002645 --- 0.1

524 FBgn0003165 --- 0.6255209445953369 seconds ---
FBgn0003169
525 FBgn0003169 --- 0.10607457160949707 seconds ---
FBgn0003174
526 FBgn0003174 --- 0.15820527076721191 seconds ---
FBgn0003175
527 FBgn0003175 --- 0.40915656089782715 seconds ---
FBgn0003178
528 FBgn0003178 --- 0.07959151268005371 seconds ---
FBgn0003187
529 FBgn0003187 --- 0.16570711135864258 seconds ---
FBgn0003189
530 FBgn0003189 --- 0.4625551700592041 seconds ---
FBgn0003204
531 FBgn0003204 --- 0.14246153831481934 seconds ---
FBgn0003205
532 FBgn0003205 --- 0.0362398624420166 seconds ---
FBgn0003206
533 FBgn0003206 --- 0.03803443908691406 seconds ---
FBgn0003209
534 FBgn0003209 --- 0.3133811950683594 seconds ---
FBgn0003210
535 FBgn0003210 --- 0.21507692337036133 seconds ---
FBgn0003218
536 FBgn0003218 --- 0.745643138885498 seconds ---
FBgn0003227
537 FBgn0003227 --- 0.12958192825317383 seconds ---
FBgn0003231
538 FBgn0003231 --- 0.09904646873474121 seconds ---
FBgn0003248
539 FBgn0003248 --- 0.0510561466217041 seconds

...

6534 FBgn0035385 --- 0.12189531326293945 seconds ---
FBgn0035388
6535 FBgn0035388 --- 0.08297204971313477 seconds ---
FBgn0035390
6536 FBgn0035390 --- 0.04103565216064453 seconds ---
FBgn0035392
6537 FBgn0035392 --- 0.14119911193847656 seconds ---
FBgn0035393
6538 FBgn0035393 --- 0.03597259521484375 seconds ---
FBgn0035397
6539 FBgn0035397 --- 0.7172942161560059 seconds ---
FBgn0035398
6540 FBgn0035398 --- 0.07269406318664551 seconds ---
FBgn0035400
6541 FBgn0035400 --- 0.3234724998474121 seconds ---
FBgn0035401
6542 FBgn0035401 --- 0.04971003532409668 seconds ---
FBgn0035402
6543 FBgn0035402 --- 0.1177682876586914 seconds ---
FBgn0035405
6544 FBgn0035405 --- 0.09071111679077148 seconds ---
FBgn0035407
6545 FBgn0035407 --- 0.07235074043273926 seconds ---
FBgn0035409
6546 FBgn0035409 --- 0.04281973838806152 seconds ---
FBgn0035410
6547 FBgn0035410 --- 0.09827184677124023 seconds ---
FBgn0035411
6548 FBgn0035411 --- 0.3808462619781494 seconds ---
FBgn0035412
6549 FBgn0035412 --- 0.034958

6661 FBgn0035577 --- 0.15500926971435547 seconds ---
FBgn0035578
6662 FBgn0035578 --- 0.06539607048034668 seconds ---
FBgn0035581
6663 FBgn0035581 --- 0.43416738510131836 seconds ---
FBgn0035582
6664 FBgn0035582 --- 0.04728507995605469 seconds ---
FBgn0035583
6665 FBgn0035583 --- 0.03584623336791992 seconds ---
FBgn0035584
6666 FBgn0035584 --- 0.03467965126037598 seconds ---
FBgn0035585
6667 FBgn0035585 --- 0.035140275955200195 seconds ---
FBgn0035586
6668 FBgn0035586 --- 0.08467245101928711 seconds ---
FBgn0035587
6669 FBgn0035587 --- 0.08475041389465332 seconds ---
FBgn0035588
6670 FBgn0035588 --- 0.04616355895996094 seconds ---
FBgn0035589
6671 FBgn0035589 --- 0.03928995132446289 seconds ---
FBgn0035590
6672 FBgn0035590 --- 0.041054725646972656 seconds ---
FBgn0035591
6673 FBgn0035591 --- 0.06471991539001465 seconds ---
FBgn0035592
6674 FBgn0035592 --- 0.03742480278015137 seconds ---
FBgn0035593
6675 FBgn0035593 --- 0.10548710823059082 seconds ---
FBgn0035594
6676 FBgn0035594 --- 0.

6788 FBgn0035776 --- 0.10653805732727051 seconds ---
FBgn0035777
6789 FBgn0035777 --- 0.07515954971313477 seconds ---
FBgn0035779
6790 FBgn0035779 --- 0.05185699462890625 seconds ---
FBgn0035780
6791 FBgn0035780 --- 0.05028820037841797 seconds ---
FBgn0035781
6792 FBgn0035781 --- 0.07871747016906738 seconds ---
FBgn0035782
6793 FBgn0035782 --- 0.04571104049682617 seconds ---
FBgn0035785
6794 FBgn0035785 --- 0.05552172660827637 seconds ---
FBgn0035786
6795 FBgn0035786 --- 0.09614849090576172 seconds ---
FBgn0035787
6796 FBgn0035787 --- 0.0508580207824707 seconds ---
FBgn0035788
6797 FBgn0035788 --- 0.044283151626586914 seconds ---
FBgn0035789
6798 FBgn0035789 --- 0.05272030830383301 seconds ---
FBgn0035790
6799 FBgn0035790 --- 0.05654716491699219 seconds ---
FBgn0035791
6800 FBgn0035791 --- 0.07728695869445801 seconds ---
FBgn0035792
6801 FBgn0035792 --- 0.04282379150390625 seconds ---
FBgn0035793
6802 FBgn0035793 --- 0.5762410163879395 seconds ---
FBgn0035795
6803 FBgn0035795 --- 0.039

6915 FBgn0035976 --- 0.3272686004638672 seconds ---
FBgn0035977
6916 FBgn0035977 --- 0.04493880271911621 seconds ---
FBgn0035978
6917 FBgn0035978 --- 0.14396381378173828 seconds ---
FBgn0035980
6918 FBgn0035980 --- 0.04356670379638672 seconds ---
FBgn0035981
6919 FBgn0035981 --- 0.07355213165283203 seconds ---
FBgn0035982
6920 FBgn0035982 --- 0.035811424255371094 seconds ---
FBgn0035983
6921 FBgn0035983 --- 0.0931859016418457 seconds ---
FBgn0035985
6922 FBgn0035985 --- 0.0384516716003418 seconds ---
FBgn0035986
6923 FBgn0035986 --- 0.1123197078704834 seconds ---
FBgn0035987
6924 FBgn0035987 --- 0.054082393646240234 seconds ---
FBgn0035988
6925 FBgn0035988 --- 0.04745984077453613 seconds ---
FBgn0035989
6926 FBgn0035989 --- 0.17908835411071777 seconds ---
FBgn0035993
6927 FBgn0035993 --- 0.047821760177612305 seconds ---
FBgn0035995
6928 FBgn0035995 --- 0.1135098934173584 seconds ---
FBgn0035996
6929 FBgn0035996 --- 0.038530826568603516 seconds ---
FBgn0035997
6930 FBgn0035997 --- 0.096

7043 FBgn0036181 --- 0.18278980255126953 seconds ---
FBgn0036183
7044 FBgn0036183 --- 0.04568147659301758 seconds ---
FBgn0036184
7045 FBgn0036184 --- 0.047480106353759766 seconds ---
FBgn0036186
7046 FBgn0036186 --- 0.07433748245239258 seconds ---
FBgn0036187
7047 FBgn0036187 --- 0.08950662612915039 seconds ---
FBgn0036188
7048 FBgn0036188 --- 0.05640363693237305 seconds ---
FBgn0036191
7049 FBgn0036191 --- 0.13721323013305664 seconds ---
FBgn0036192
7050 FBgn0036192 --- 0.03543519973754883 seconds ---
FBgn0036193
7051 FBgn0036193 --- 0.058388471603393555 seconds ---
FBgn0036194
7052 FBgn0036194 --- 0.10155487060546875 seconds ---
FBgn0036195
7053 FBgn0036195 --- 0.08840560913085938 seconds ---
FBgn0036196
7054 FBgn0036196 --- 0.13232636451721191 seconds ---
FBgn0036198
7055 FBgn0036198 --- 0.0345158576965332 seconds ---
FBgn0036199
7056 FBgn0036199 --- 0.06685709953308105 seconds ---
FBgn0036202
7057 FBgn0036202 --- 0.0733175277709961 seconds ---
FBgn0036203
7058 FBgn0036203 --- 0.10

Cleaning positions degenerancy based on most constrain posibility

In [ ]:
df = pd.read_csv(DATA + '/annotations/degeneracyDmelPositions.tab',sep='\t',header=None,names=['CHROM','POS','degen'])
    
chrList = ['2L','2R','3L','3R','X']
import time
for nchr in chrList:
    start_time = time.time()
    print(nchr)
    tmp = df[df['CHROM'] == nchr]
    tmp = tmp.groupby(['CHROM','POS']).agg({'degen':','.join}).reset_index()
    tmp['type'] = np.nan

    tmp['type'] = tmp['degen'].apply(lambda row: foldPositions(row))
    tmp = tmp.sort_values('POS')
    
    tmp.to_csv(DATA + '/annotations/zeroFourFoldPositions.tab',header=False,index=False,mode='a',sep='\t')
    print("--- %s seconds ---" % (time.time() - start_time))

#### Undetermined nucleotide distributions by cds position

In [ ]:
dfGenes = pd.read_csv(DATA + '/annotations/flybaseGenesCleaned.tab',header = 0,sep='\t')
cds = pd.read_csv(DATA + '/annotations/cdsCoordinates.tab',header = 0,sep='\t')
cds = pd.merge(cds, dfGenes,  how='inner', left_on=['chr','name'], right_on = ['chr','name'])
cds = cds.loc[cds.reset_index().groupby(['chr','id'])['transcriptSize'].idxmax()].reset_index(drop=True)

Raleigh

In [ ]:
nDistribution(cds,'RAL')

Zambia

In [ ]:
nDistribution(cds,'ZI')

Retrieve by population the number of N

In [ ]:
for pop in ['RAL','ZI']:
    print(pop)
    
    df = pd.read_csv(DATA + '/nCall/ncall' + pop + '.tab',sep='\t',header=None)
    nCalls = df[0].value_counts().reset_index()
    
    nCalls = pd.DataFrame(nCalls)
    nCalls = nCalls.sort_values('index')
    
    nCalls['freq'] = nCalls[0].apply(lambda x: x/sum(nCalls[0])*100)
    
    nCalls.columns = ['positions','count','freq']
    nCalls.to_csv(DATA + '/nCall/' + pop + 'DistributionN.tab',sep='\t',header=True,index=False)

### Extracting Derived Allele Frequency and Divergence by population and type of site

Opening cdsCoordinates file to extract fasta sequence using pyfaidx

In [ ]:
!python /home/jmurga/mktComparison/scripts/src/sfsDivergence.py --help

#### *D. simulans*

##### Raleigh population

In [ ]:
!python /home/jmurga/mktComparison/scripts/src/sfsDivergence.py --genes flybaseGenesCleaned.tab --cds cdsCoordinates.tab --population RAL --sampling 160 --outgroup dsim

##### Zambia population

In [ ]:
!python /home/jmurga/mktComparison/scripts/src/sfsDivergence.py --genes flybaseGenesCleaned.tab --cds cdsCoordinates.tab --population ZI --sampling 154 --outgroup dsim

#### *D. yakuba*

##### Raleigh population

In [ ]:
!python /home/jmurga/mktComparison/scripts/src/sfsDivergence.py --genes flybaseGenesCleaned.tab --cds cdsCoordinates.tab --population RAL --sampling 160 --outgroup dyak

##### Zambia population

In [ ]:
!python /home/jmurga/mktComparison/scripts/src/sfsDivergence.py --genes flybaseGenesCleaned.tab --cds cdsCoordinates.tab --population ZI --sampling 154 --outgroup dyak

### Estimating metrics

#### Extracting mi and m0

In [2]:
dfGenes = pd.read_csv('/home/jmurga/mktComparison/rawData/dmel/annotations/flybaseGenesCleaned.tab',header = 0,usecols=['id','name','chr','strand'],sep='\t')
cds = pd.read_csv('/home/jmurga/mktComparison/rawData/dmel/annotations/cdsLargest.tab',header=0,sep='\t')
cds = pd.merge(cds, dfGenes,  how='inner', left_on=['chr','name'], right_on = ['chr','name'])
degeneratePositions = pd.read_csv('/home/jmurga/mktComparison/rawData/dmel/annotations/zeroFourFoldPositions.tab',header=None,names=['CHROM','POS','degen','type'],sep='\t')

In [ ]:
# Rewrite file each execution
columns = ['id','mi','m0']
totalFoldPositions = pd.DataFrame(columns=columns)
for j in cds['chr'].unique():
    print(j)
    chrPositions = degeneratePositions[degeneratePositions['chr']==j]
    for index, row in cds[cds['chr']==j].iterrows():
#     for index, row in cds[cds['id']=='FBgn0000017'].iterrows():
        print(index,row['id'])
        chrFile = Fasta('/data/shared/dgn/ref/Chr' + row['chr'] +'.fasta')
    #     Convert CDS list into numeric array
        coordinates = array(row['coordinates'].split(',')).astype(int).tolist()
        coordinates =  [coordinates[i:i+2] for i in range(0, len(coordinates), 2)]
        # Extract all CDS positions in a list in order to merge with degenerate sequences (same length -> same index)
        positions=[]
        for i in range(0,len(coordinates),1):
            positions.append(list(range(coordinates[i][0],coordinates[i][1]+1)))  
        allPositions = [item for sublist in positions for item in sublist]
        # Extract cds sequences
        seq = chrFile.get_spliced_seq(row['chr'].replace('chr',''), coordinates).seq.upper()
    #     print(len(seq))
        if(row['strand'] == '-'):
            seq = reverseComplement(seq)
            allPositions = allPositions[::-1]
        if((len(seq)/3).is_integer() and seq[0:3]=='ATG'):
            m = degenerate(seq)
            tmp = pd.DataFrame({'POS':allPositions,'m':list(m)})
            tmp['chr']=row['chr']
            tmp = pd.merge(chrPositions,tmp,on=['chr','POS'],how='right')              
            tmp['m']=tmp.apply(lambda x: '0' if x['newType']=='0fold' else '4' if x['newType']=='4fold' else x['m'],axis=1)
            counts = tmp['m'].value_counts()
            if(counts.shape[0]<4 or '4' not in counts.index):
                m0=0
                mi=counts['0']
            else:
                m0=counts['4']
                mi=counts['0']
            data = pd.DataFrame({'id':row['id'],'mi':mi,'m0':m0},index=[0])
            data.to_csv('/home/jmurga/mktComparison/rawData/refAnalyzableSites.tab',sep='\t',header=False,mode='a',index=False)
#             totalFoldPositions = totalFoldPositions.append(data,ignore_index=True)
        else:
            data = pd.DataFrame({'id':row['id'],'mi':0,'m0':0},index=[0])
            data.to_csv('/home/jmurga/mktComparison/rawData/refAnalyzableSites.tab',sep='\t',header=False,mode='a',index=False)
#             totalFoldPositions = totalFoldPositions.append(data,ignore_index=True)

####  *D. simulans*

##### Extract by population Derived Allele Frequency and Divergence in a file to format by functional class and populations

In [12]:
degeneratePositions = pd.read_csv('/home/jmurga/mktComparison/rawData/dmel/annotations/zeroFourFoldPositions.tab',header=None,names=['CHROM','POS','degen','type'],sep='\t')

In [15]:
pops=['RAL','ZI']    

for p in pops:
    print(p)
    popSites = pd.read_csv(DATA + '/alleleFrequencies/dsim/' + 'dsimDmelSites' + p + '.tab',sep='\t',header=None,names=['id','chr','POS','div','rawDerivedAllele','type','pop'])
        
    popSites.to_csv(DATA+ '/alleleFrequencies/dsim/dsimDmelSites.tab',sep='\t',index=False,header=False,mode='a')

RAL
ZI


In [19]:
dmelSites = pd.read_csv(DATA+ '/alleleFrequencies/dsim/dsimDmelSites.tab', sep='\t', header=None)
dmelSites.columns = ['id','CHROM','POS','div','rawDerivedAllele','type','pop']
dmelSites = dmelSites.drop('type',axis=1)
dmelSites.head()

,id,CHROM,POS,div,rawDerivedAllele,pop
0,FBgn0000018,2L,10975207,1,0.00000,RAL
1,FBgn0000018,2L,10975203,1,0.00000,RAL
2,FBgn0000018,2L,10975191,1,0.00000,RAL
3,FBgn0000018,2L,10975141,0,0.41875,RAL
4,FBgn0000018,2L,10975091,1,0.00000,RAL


In [20]:
dmelSites = pd.merge(dmelSites,degeneratePositions,how='left',left_on=['CHROM','POS'],right_on=['CHROM','POS'])
dmelSites = dmelSites[dmelSites['type']!='mixed']
dmelSites.head()

,id,CHROM,POS,div,rawDerivedAllele,pop,degen,type
0,FBgn0000018,2L,10975207,1,0.00000,RAL,4,4fold
1,FBgn0000018,2L,10975203,1,0.00000,RAL,0,0fold
2,FBgn0000018,2L,10975191,1,0.00000,RAL,0,0fold
3,FBgn0000018,2L,10975141,0,0.41875,RAL,4,4fold
4,FBgn0000018,2L,10975091,1,0.00000,RAL,0,0fold


##### Divergence

In [21]:
div = dmelSites.groupby(['id','type','pop'])['div'].sum().reset_index()
div = div.pivot_table(index=['id','pop'],columns=['type'],values='div').reset_index()
div.columns = ['id','pop','di','d0']

In [22]:
div.head()

,id,pop,di,d0
0,FBgn0000008,RAL,27.0,31.0
1,FBgn0000008,ZI,26.0,31.0
2,FBgn0000014,RAL,1.0,20.0
3,FBgn0000014,ZI,1.0,18.0
4,FBgn0000015,RAL,0.0,3.0


##### Derived Allele Frequency

In [23]:
daf = dmelSites[['id','rawDerivedAllele','type','pop']][dmelSites['rawDerivedAllele']!=0]

bins = np.arange(0,1.05,0.05)
labels =  np.arange(0.05,1.05,0.05)

daf['categories'] = pd.cut(daf['rawDerivedAllele'],bins=bins,labels=labels)

sfs = daf.groupby(['id','type','categories','pop']).count().reset_index()
sfs['rawDerivedAllele'] = sfs['rawDerivedAllele'].fillna(0).astype(int)
sfs = sfs.groupby(['id','pop','type'])['rawDerivedAllele'].apply(list).reset_index()

sfs = sfs.pivot_table(index=['id','pop'], columns=['type'],values='rawDerivedAllele',aggfunc=lambda x:x).reset_index()
sfs['p0'] = sfs['4fold'].apply(lambda x: sum(x))
sfs['pi'] = sfs['0fold'].apply(lambda x: sum(x))

sfs['0fold'] = sfs['0fold'].apply(lambda x:';'.join(map(str,x)))
sfs['4fold'] = sfs['4fold'].apply(lambda x:';'.join(map(str,x)))

sfs.columns = ['id','pop','daf0f','daf4f','p0','pi']

In [25]:
sfs.head()

,id,pop,daf0f,daf4f,p0,pi
0,FBgn0000008,RAL,30;7;4;0;1;0;0;0;1;1;0;0;1;0;0;0;0;0;0;0,24;5;3;3;0;0;1;1;1;0;1;1;0;1;0;0;1;0;0;0,42,45
1,FBgn0000008,ZI,67;2;2;3;0;0;0;1;0;0;0;1;0;0;0;0;0;0;0;0,55;7;4;3;2;3;1;4;0;1;0;0;0;0;1;0;0;1;0;0,82,76
2,FBgn0000014,RAL,2;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0,1;0;1;1;0;0;0;0;0;0;0;0;0;0;0;0;0;0;1;0,4,2
3,FBgn0000014,ZI,4;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0,10;0;0;1;0;1;0;0;0;0;0;0;0;1;0;1;0;0;0;2,16,4
4,FBgn0000015,RAL,3;0;0;0;0;1;0;0;0;0;0;0;0;0;0;0;0;0;0;0,5;0;3;1;0;1;0;0;0;0;1;0;0;1;1;0;0;0;0;0,13,4


##### Merging info

In [27]:
mktData = pd.merge(sfs,div,on=['id','pop'],how='outer')

In [28]:
mktData.head()

,id,pop,daf0f,daf4f,p0,pi,di,d0
0,FBgn0000008,RAL,30;7;4;0;1;0;0;0;1;1;0;0;1;0;0;0;0;0;0;0,24;5;3;3;0;0;1;1;1;0;1;1;0;1;0;0;1;0;0;0,42.0,45.0,27.0,31.0
1,FBgn0000008,ZI,67;2;2;3;0;0;0;1;0;0;0;1;0;0;0;0;0;0;0;0,55;7;4;3;2;3;1;4;0;1;0;0;0;0;1;0;0;1;0;0,82.0,76.0,26.0,31.0
2,FBgn0000014,RAL,2;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0,1;0;1;1;0;0;0;0;0;0;0;0;0;0;0;0;0;0;1;0,4.0,2.0,1.0,20.0
3,FBgn0000014,ZI,4;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0,10;0;0;1;0;1;0;0;0;0;0;0;0;1;0;1;0;0;0;2,16.0,4.0,1.0,18.0
4,FBgn0000015,RAL,3;0;0;0;0;1;0;0;0;0;0;0;0;0;0;0;0;0;0;0,5;0;3;1;0;1;0;0;0;0;1;0;0;1;1;0;0;0;0;0,13.0,4.0,0.0,3.0


In [29]:
dfGenes = pd.read_csv(DATA + '/annotations/flybaseGenesCleaned.tab',sep='\t',header=0)
pops=['RAL','ZI']                                   
genesByPop = pd.DataFrame()

for p in pops:
    
    dfGenes['pop'] = p
    genesByPop = genesByPop.append(dfGenes)
    
genesByPop = genesByPop.reset_index(drop=True)

In [34]:
pops=['RAL','ZI']                                   
refAnalizableSites = pd.read_csv(DATA+'/refAnalyzableSites.tab',sep='\t',header=0)

totalFoldPositionsByPop = pd.DataFrame()
for p in pops:
    print(p)
    refAnalizableSites['pop'] = p
    
    totalFoldPositionsByPop = totalFoldPositionsByPop.append(refAnalizableSites)

totalFoldPositionsByPop = totalFoldPositionsByPop.reset_index(drop=True)

RAL
ZI


In [35]:
genesByPop = pd.merge(genesByPop,totalFoldPositionsByPop,on=['id','pop'],how='outer')

In [36]:
mktData = pd.merge(mktData,genesByPop,on=['id','pop'],how='outer')
mktData[mktData['daf0f'].isna()].loc[:,'daf0f'] = '0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0'
mktData[mktData['daf4f'].isna()].loc[:,'daf4f'] = '0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0'
mktData['daf0f'] = mktData['daf0f'].fillna('0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0')
mktData['daf4f'] = mktData['daf4f'].fillna('0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0')
mktData = mktData.fillna(0)

/home/jmurga/.conda/envs/proteins/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [37]:
dfRecomb = pd.read_csv(DATA + '/genesRecombination.tab',sep='\t')
dfRecomb.columns = ['id','recomb']
mktData = pd.merge(mktData,dfRecomb,on=['id'])

In [40]:
mktData = mktData[['id', 'pop', 'daf0f', 'daf4f', 'p0', 'pi', 'di', 'd0', 'chr','mi','m0','recomb']]

In [41]:
mktData.head()

,id,pop,daf0f,daf4f,p0,pi,di,d0,chr,mi,m0,recomb
0,FBgn0000008,RAL,30;7;4;0;1;0;0;0;1;1;0;0;1;0;0;0;0;0;0;0,24;5;3;3;0;0;1;1;1;0;1;1;0;1;0;0;1;0;0;0,42.0,45.0,27.0,31.0,2R,2539,667,2.169284
1,FBgn0000008,ZI,67;2;2;3;0;0;0;1;0;0;0;1;0;0;0;0;0;0;0;0,55;7;4;3;2;3;1;4;0;1;0;0;0;0;1;0;0;1;0;0,82.0,76.0,26.0,31.0,2R,2539,667,2.169284
2,FBgn0000014,RAL,2;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0,1;0;1;1;0;0;0;0;0;0;0;0;0;0;0;0;0;0;1;0,4.0,2.0,1.0,20.0,3R,1153,292,0.761699
3,FBgn0000014,ZI,4;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0,10;0;0;1;0;1;0;0;0;0;0;0;0;1;0;1;0;0;0;2,16.0,4.0,1.0,18.0,3R,1153,292,0.761699
4,FBgn0000015,RAL,3;0;0;0;0;1;0;0;0;0;0;0;0;0;0;0;0;0;0;0,5;0;3;1;0;1;0;0;0;0;1;0;0;1;1;0;0;0;0;0,13.0,4.0,0.0,3.0,3R,962,232,0.435257


In [42]:
mktData.to_csv('/home/jmurga/mktComparison/results/dsimDmelData.tab',sep='\t',header=True,index=False)

#### *D. yakuba*

##### Extract by population Derived Allele Frequency and Divergence in a file to format by functional class and populations

In [12]:
degeneratePositions = pd.read_csv('/home/jmurga/mktComparison/rawData/dmel/annotations/zeroFourFoldPositions.tab',header=None,names=['CHROM','POS','degen','type'],sep='\t')

In [44]:
pops=['RAL','ZI']    

for p in pops:
    print(p)
    popSites = pd.read_csv(DATA + '/alleleFrequencies/dyak/' + 'dyakDmelSites' + p + '.tab',sep='\t',header=None,names=['id','chr','POS','div','rawDerivedAllele','type','pop'])
        
    popSites.to_csv(DATA+ '/alleleFrequencies/dyak/dyakDmelSites.tab',sep='\t',index=False,header=False,mode='a')

RAL
ZI


In [66]:
dmelSites = pd.read_csv(DATA+ '/alleleFrequencies/dyak/dyakDmelSites.tab', sep='\t', header=None)
dmelSites.columns = ['id','CHROM','POS','div','rawDerivedAllele','type','pop']
dmelSites = dmelSites.drop('type',axis=1)
dmelSites.head()

,id,CHROM,POS,div,rawDerivedAllele,pop
0,FBgn0000018,2L,10975203,1,0.0,RAL
1,FBgn0000018,2L,10975192,1,0.0,RAL
2,FBgn0000018,2L,10975190,1,0.0,RAL
3,FBgn0000018,2L,10975141,0,0.4,RAL
4,FBgn0000018,2L,10975131,1,0.0,RAL


In [67]:
dmelSites = pd.merge(dmelSites,degeneratePositions,how='left',left_on=['CHROM','POS'],right_on=['CHROM','POS'])
dmelSites = dmelSites[dmelSites['type']!='mixed']
dmelSites.head()

,id,CHROM,POS,div,rawDerivedAllele,pop,degen,type
0,FBgn0000018,2L,10975203,1,0.0,RAL,0,0fold
1,FBgn0000018,2L,10975192,1,0.0,RAL,4,4fold
2,FBgn0000018,2L,10975190,1,0.0,RAL,0,0fold
3,FBgn0000018,2L,10975141,0,0.4,RAL,4,4fold
4,FBgn0000018,2L,10975131,1,0.0,RAL,0,0fold


##### Divergence

In [68]:
div = dmelSites.groupby(['id','type','pop'])['div'].sum().reset_index()
div = div.pivot_table(index=['id','pop'],columns=['type'],values='div').reset_index()
div.columns = ['id','pop','di','d0']

In [69]:
div.head()

,id,pop,di,d0
0,FBgn0000008,RAL,94.0,75.0
1,FBgn0000008,ZI,93.0,68.0
2,FBgn0000014,RAL,0.0,16.0
3,FBgn0000014,ZI,0.0,15.0
4,FBgn0000015,RAL,2.0,14.0


##### Derived Allele Frequency

In [70]:
daf = dmelSites[['id','rawDerivedAllele','type','pop']][dmelSites['rawDerivedAllele']!=0]

bins = np.arange(0,1.05,0.05)
labels =  np.arange(0.05,1.05,0.05)

daf['categories'] = pd.cut(daf['rawDerivedAllele'],bins=bins,labels=labels)

sfs = daf.groupby(['id','type','categories','pop']).count().reset_index()
sfs['rawDerivedAllele'] = sfs['rawDerivedAllele'].fillna(0).astype(int)
sfs = sfs.groupby(['id','pop','type'])['rawDerivedAllele'].apply(list).reset_index()

sfs = sfs.pivot_table(index=['id','pop'], columns=['type'],values='rawDerivedAllele',aggfunc=lambda x:x).reset_index()
sfs['p0'] = sfs['4fold'].apply(lambda x: sum(x))
sfs['pi'] = sfs['0fold'].apply(lambda x: sum(x))

sfs['0fold'] = sfs['0fold'].apply(lambda x:';'.join(map(str,x)))
sfs['4fold'] = sfs['4fold'].apply(lambda x:';'.join(map(str,x)))

sfs.columns = ['id','pop','daf0f','daf4f','p0','pi']

In [71]:
sfs.head()

,id,pop,daf0f,daf4f,p0,pi
0,FBgn0000008,RAL,26;6;4;2;0;0;1;0;1;0;1;0;0;0;0;0;0;0;0;2,21;7;2;1;0;0;1;1;1;0;2;1;0;0;0;0;0;1;0;3,41,43
1,FBgn0000008,ZI,66;2;1;3;0;0;0;1;1;0;0;0;0;0;0;0;0;0;0;4,49;6;4;4;1;0;4;3;0;1;0;0;0;0;0;0;0;2;0;6,80,78
2,FBgn0000014,RAL,2;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0,1;0;1;1;0;0;0;0;0;0;0;0;0;0;0;0;0;0;1;0,4,2
3,FBgn0000014,ZI,5;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0,9;0;0;1;1;0;0;0;0;0;0;0;0;0;1;0;1;0;0;0,13,5
4,FBgn0000015,RAL,4;0;0;0;0;0;1;0;0;0;0;0;0;0;0;0;0;0;0;0,4;0;3;0;1;0;1;0;0;0;1;0;1;0;1;0;0;0;0;0,12,5


##### Merging info

In [72]:
mktData = pd.merge(sfs,div,on=['id','pop'],how='outer')

In [73]:
mktData.head()

,id,pop,daf0f,daf4f,p0,pi,di,d0
0,FBgn0000008,RAL,26;6;4;2;0;0;1;0;1;0;1;0;0;0;0;0;0;0;0;2,21;7;2;1;0;0;1;1;1;0;2;1;0;0;0;0;0;1;0;3,41.0,43.0,94.0,75.0
1,FBgn0000008,ZI,66;2;1;3;0;0;0;1;1;0;0;0;0;0;0;0;0;0;0;4,49;6;4;4;1;0;4;3;0;1;0;0;0;0;0;0;0;2;0;6,80.0,78.0,93.0,68.0
2,FBgn0000014,RAL,2;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0,1;0;1;1;0;0;0;0;0;0;0;0;0;0;0;0;0;0;1;0,4.0,2.0,0.0,16.0
3,FBgn0000014,ZI,5;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0,9;0;0;1;1;0;0;0;0;0;0;0;0;0;1;0;1;0;0;0,13.0,5.0,0.0,15.0
4,FBgn0000015,RAL,4;0;0;0;0;0;1;0;0;0;0;0;0;0;0;0;0;0;0;0,4;0;3;0;1;0;1;0;0;0;1;0;1;0;1;0;0;0;0;0,12.0,5.0,2.0,14.0


In [74]:
dfGenes = pd.read_csv(DATA + '/annotations/flybaseGenesCleaned.tab',sep='\t',header=0)
pops=['RAL','ZI']                                   
genesByPop = pd.DataFrame()

for p in pops:
    
    dfGenes['pop'] = p
    genesByPop = genesByPop.append(dfGenes)
    
genesByPop = genesByPop.reset_index(drop=True)

In [75]:
pops=['RAL','ZI']                                   
refAnalizableSites = pd.read_csv(DATA+'/refAnalyzableSites.tab',sep='\t',header=0)

totalFoldPositionsByPop = pd.DataFrame()
for p in pops:
    print(p)
    refAnalizableSites['pop'] = p
    
    totalFoldPositionsByPop = totalFoldPositionsByPop.append(refAnalizableSites)

totalFoldPositionsByPop = totalFoldPositionsByPop.reset_index(drop=True)

RAL
ZI


In [76]:
genesByPop = pd.merge(genesByPop,totalFoldPositionsByPop,on=['id','pop'],how='outer')

In [77]:
mktData = pd.merge(mktData,genesByPop,on=['id','pop'],how='outer')
mktData[mktData['daf0f'].isna()].loc[:,'daf0f'] = '0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0'
mktData[mktData['daf4f'].isna()].loc[:,'daf4f'] = '0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0'
mktData['daf0f'] = mktData['daf0f'].fillna('0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0')
mktData['daf4f'] = mktData['daf4f'].fillna('0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0')
mktData = mktData.fillna(0)

In [78]:
dfRecomb = pd.read_csv(DATA + '/genesRecombination.tab',sep='\t')
dfRecomb.columns = ['id','recomb']
mktData = pd.merge(mktData,dfRecomb,on=['id'])

In [79]:
mktData = mktData[['id', 'pop', 'daf0f', 'daf4f', 'p0', 'pi', 'di', 'd0', 'chr','mi','m0','recomb']]

In [80]:
mktData.head()

,id,pop,daf0f,daf4f,p0,pi,di,d0,chr,mi,m0,recomb
0,FBgn0000008,RAL,26;6;4;2;0;0;1;0;1;0;1;0;0;0;0;0;0;0;0;2,21;7;2;1;0;0;1;1;1;0;2;1;0;0;0;0;0;1;0;3,41.0,43.0,94.0,75.0,2R,2539,667,2.169284
1,FBgn0000008,ZI,66;2;1;3;0;0;0;1;1;0;0;0;0;0;0;0;0;0;0;4,49;6;4;4;1;0;4;3;0;1;0;0;0;0;0;0;0;2;0;6,80.0,78.0,93.0,68.0,2R,2539,667,2.169284
2,FBgn0000014,RAL,2;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0,1;0;1;1;0;0;0;0;0;0;0;0;0;0;0;0;0;0;1;0,4.0,2.0,0.0,16.0,3R,1153,292,0.761699
3,FBgn0000014,ZI,5;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0;0,9;0;0;1;1;0;0;0;0;0;0;0;0;0;1;0;1;0;0;0,13.0,5.0,0.0,15.0,3R,1153,292,0.761699
4,FBgn0000015,RAL,4;0;0;0;0;0;1;0;0;0;0;0;0;0;0;0;0;0;0;0,4;0;3;0;1;0;1;0;0;0;1;0;1;0;1;0;0;0;0;0,12.0,5.0,2.0,14.0,3R,962,232,0.435257


In [81]:
mktData[mktData['pop']=='RAL'].d0.sum()/mktData[mktData['pop']=='RAL'].m0.sum()

0.14907746440928907

In [82]:
mktData.to_csv('/home/jmurga/mktComparison/results/dyakDmelData.tab',sep='\t',header=True,index=False)